# Knot ID

## Setup

### Libraries

In [ ]:
from torchvision import transforms
from torchvision.datasets.vision import VisionDataset
from torch.utils.data import random_split, DataLoader
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import os

### Constants

In [ ]:
batch_size = 64
img_size = 32
num_classes = 10
num_epochs = 1
test_split = 0.2
train_split = 1.0 - test_split
learning_rate = 1e-2
crop_size = 3456

## Model

### Dataset

In [ ]:
class Knots(VisionDataset):

    def __init__(self, transform=None):
        self.root = './data/'
        self.transform = transform
        self.filepaths = []
        self.targets = []
        self.classes = {}

        super().__init__(self.root, transforms=None, transform=transform)

        class_idx = 0
        for filename in os.listdir(os.path.join(self.root, '10Knots')):
            if filename != '.DS_Store':
                self.classes[class_idx] = filename
                class_idx += 1
        
        for idx, label in self.classes.items():
            for path, _, filenames in os.walk(os.path.join(self.root, '10Knots', label)):
                for filename in filenames:
                    if filename != '.DS_Store':
                        self.filepaths.append(os.path.join(path, filename))
                        self.targets.append(idx)

    def __len__(self):
        return len(self.filepaths)
    
    def __getitem__(self, idx):
        img = Image.open(self.filepaths[idx])
        target = self.targets[idx]

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def get_class(self, idx):
        return self.classes[idx]

### Model

In [ ]:
class KnotClassifier(nn.Module):

    def __init__(self):
        super(KnotClassifier, self).__init__()

        # input shape (64, 3, 32, 32)
        self.feature_learning = nn.Sequential(
            nn.Conv2d(3, 10, 3, 1, 1),      # (64, 10, 32, 32)
            nn.ReLU(),
            nn.MaxPool2d(2),                # (64, 10, 16, 16)
            nn.Conv2d(10, 20, 3, 1, 1),     # (64, 20, 16, 16)
            nn.ReLU(),
            nn.MaxPool2d(2),                # (64, 20, 8, 8)
        )

        self.classification = nn.Sequential(
            nn.Flatten(1),                  # (64, 1280)
            nn.Linear(1280, 256),           # (64, 256)
            nn.ReLU(),
            nn.Linear(256,64),              # (64, 64)
            nn.ReLU(),
            nn.Linear(64,10)                # (64, 10)
        )

    def forward(self, x):
        x = self.feature_learning(x)
        x = self.classification(x)
        return x

## Train and test functions

In [ ]:
def train(model, train_loader, loss_fn, optimizer, epoch):
    model.train()

    for batch_idx, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output, targets)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(
                f'Epoch {epoch}: [{batch_idx*len(images)}/{len(train_loader.dataset)}]'
                f'Loss: {loss.item():.4f}'
            )

In [ ]:
def test(model, test_loader, loss_fn, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for images, targets in test_loader:
            output = model(images)
            test_loss += loss_fn(output, targets, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(targets.data.view_as(pred)).sum()
            test_loss /= len(test_loader.dataset)
            print(
                f'Test result on epoch {epoch}: '
                f'Avg loss is {test_loss:.4f}, '
                f'Accuracy: {(100.0 * correct / len(test_loader.dataset)):.2f}%'
            )

## Training and testing

In [ ]:
transform = transforms.Compose([
    transforms.CenterCrop(crop_size),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.7019, 0.4425, 0.1954), (0.1720, 0.1403, 0.1065))
])

train_data, test_data = random_split(Knots(transform=transform), [train_split, test_split])
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

model = KnotClassifier()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, num_epochs+1):
    train(model, train_loader, loss_fn, optimizer, epoch)
    test(model, test_loader, loss_fn, epoch)